In [5]:
import pandas as pd
import requests
import os
import time

matches = pd.read_csv('WCMatches.csv')
matches.columns
matches[["winnerid","loserid"]]




,winnerid,loserid
0,7d05pfwn,jfvflnna
1,o8q1tnkp,o8q1tnkp
2,akq5t76h,akq5t76h
3,ng9mpb4s,ng9mpb4s
4,ng9mpb4s,ng9mpb4s
...,...,...
2783,dq44xncy,222vn7x7
2784,g7umra42,g7umra42
2785,g7umra42,222vn7x7
2786,g7umra42,dq44xncy


In [11]:
# Ensure unique_ids is unique
unique_ids = pd.concat([matches['winnerid'], matches['loserid']]).unique()
print(unique_ids.size)
player_info = pd.DataFrame()

1210


The following is only for one time execution to get the first info of all players from the inofficial WC

In [12]:
import os
# Load existing player_info if it exists, otherwise create empty DataFrame
player_info_path = "EloPlayerinfo.csv"
if os.path.exists(player_info_path):
    player_info = pd.read_csv(player_info_path, low_memory=False)
    processed_ids = set(player_info['player_id'].unique())
else:
    player_info = pd.DataFrame()
    processed_ids = set()

# Only process player_ids that are not already in player_info
to_process = [pid for pid in unique_ids if pid not in processed_ids]

for idx, player_id in enumerate(to_process, start=len(processed_ids)):
    url_events = f"https://mtgeloproject.net/api/players/{player_id}/events"
    url_matches = f"https://mtgeloproject.net/api/players/{player_id}/matches"
    response_events = requests.get(url_events)
    response_matches = requests.get(url_matches)
    
    # Convert responses to JSON
    events_data = response_events.json()
    matches_data = response_matches.json()

    #flatten and rename events
    events_data_df = pd.DataFrame(events_data['data'])
    events_data_df = events_data_df.rename(columns={"code":"tournament_code"})

    # Flatten matches_data so each match is a row, including tournament code
    all_matches = []
    for tournament_code, matches_list in matches_data.items():
        for match in matches_list:
            match_copy = match.copy()
            match_copy['tournament_code'] = tournament_code
            all_matches.append(match_copy)
    matches_data_flat_df = pd.DataFrame(all_matches)

    # Expand own_elo and opp_data columns into separate columns and drop duplicate column
    matches_data_flat_df = matches_data_flat_df.join(pd.json_normalize(matches_data_flat_df.pop('own_elo')).add_prefix('own_elo_'))
    matches_data_flat_df = matches_data_flat_df.join(pd.json_normalize(matches_data_flat_df.pop('opp_data')).add_prefix('opp_'))
    matches_data_flat_df = matches_data_flat_df.drop(columns = "format")

    # Merge matches and events on event code
    merged_df = pd.merge(matches_data_flat_df, events_data_df, on="tournament_code", how="left", copy=False)
    #add player id
    merged_df.insert(loc=0, column= "player_id", value=player_id)

    # Append merged_df to player_info
    player_info = pd.concat([player_info, merged_df], ignore_index=True)

    print(f"Index: {idx}, Player ID: {player_id}")
    time.sleep(0.5) #wait 0.5 second to not pummel the server

# Save player_info after all data is collected
player_info.to_csv("EloPlayerinfo.csv", index=False)
print("List done")

List done


In [74]:
#find if new id's that have not been in the list before have come up
player_info_unique_opps = player_info[["opp_id", "opp_opp"]].drop_duplicates()
matches_unique_players = matches[["winnerid","Target"]].drop_duplicates()
matches_unique_opps = matches[["loserid","Source"]].drop_duplicates()

# Combine all into a single DataFrame with columns: id, name
all_pairs = pd.concat([
    player_info_unique_opps.rename(columns={"opp_id": "id", "opp_opp": "name"}),
    matches_unique_players.rename(columns={"winnerid": "id", "Target": "name"}),
   matches_unique_opps.rename(columns={"loserid": "id", "Source": "name"})
], ignore_index=True)

# Drop duplicates and rows with missing names or ids
all_pairs = all_pairs.dropna(subset=["id", "name"]).drop_duplicates()

# Save the result
all_pairs.to_csv("UniqueIdNamePairs.csv", index=False)


unique_ids_update = pd.DataFrame(all_pairs["id"].unique())
print(unique_ids_update.size)
unique_ids_update.to_csv("UniqueIds.csv", index=False)



90464


This is to make it repeatable

In [26]:

def get_Player_Events_Matches_from_URL(player_id):
    url_events = f"https://mtgeloproject.net/api/players/{player_id}/events"
    url_matches = f"https://mtgeloproject.net/api/players/{player_id}/matches"
    response_events = requests.get(url_events)
    response_matches = requests.get(url_matches)
    
    # Convert responses to JSON
    events_data = response_events.json()
    matches_data = response_matches.json()

    #flatten and rename events
    events_data_df = pd.DataFrame(events_data['data'])
    events_data_df = events_data_df.rename(columns={"code":"tournament_code"})

    # Flatten matches_data so each match is a row, including tournament code
    all_matches = []
    for tournament_code, matches_list in matches_data.items():
        for match in matches_list:
            match_copy = match.copy()
            match_copy['tournament_code'] = tournament_code
            all_matches.append(match_copy)
    matches_data_flat_df = pd.DataFrame(all_matches)

    # Expand own_elo and opp_data columns into separate columns and drop duplicate column
    matches_data_flat_df = matches_data_flat_df.join(pd.json_normalize(matches_data_flat_df.pop('own_elo')).add_prefix('own_elo_'))
    matches_data_flat_df = matches_data_flat_df.join(pd.json_normalize(matches_data_flat_df.pop('opp_data')).add_prefix('opp_'))
    matches_data_flat_df = matches_data_flat_df.drop(columns = "format")

    # Merge matches and events on event code
    merged_df = pd.merge(matches_data_flat_df, events_data_df, on="tournament_code", how="left", copy=False)
    #print(f"Player ID: {player_id} Events and Matches collected")
    return merged_df

In [27]:

import requests
from bs4 import BeautifulSoup
import json

def get_player_data_from_url(id):
    """
    Fetches HTML from a URL, finds the correct astro-island element,
    and extracts player data from its props attribute.
    """
    url = f"https://mtgeloproject.net/profile/{id}"
    try:
        # Fetch the HTML content from the URL
        response = requests.get(url)
        response.raise_for_status() # Raise an exception for bad status codes

        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find the astro-island element that corresponds to the Profile component.
        # We'll use the 'component-url' attribute to find it.
        # You might need to adjust the component_url value if it changes.
        # Alternatively, you could use 'component-export': 'default'
        # or look for a unique data attribute if available.
        component_url_to_find = '/_astro/Profile.FIYvQsk3.js'
        astro_island_element = soup.find('astro-island', {'component-url': component_url_to_find})


        if astro_island_element:
            props_attribute = astro_island_element.get('props')
            if props_attribute:
                try:
                    # The props attribute value is a JSON string
                    # In the provided data, the structure seems to be a list where the second element
                    # contains the 'info' object.
                    props_data = json.loads(props_attribute)

                    # Accessing the info object. Based on your provided data structure,
                    # the 'info' object is the second element in the main list value
                    # associated with the key "info".
                    info_data = props_data.get('info', [None, {}])[1]

                    player_id = info_data.get('id', [None, None])[1]
                    first_name = info_data.get('first_name', [None, None])[1]
                    last_name = info_data.get('last_name', [None, None])[1]

                    #print(f"ID: {player_id}")
                    #print(f"First Name: {first_name}")
                    #print(f"Last Name: {last_name}")
                    info = [player_id,last_name,first_name]
                    return info
                    

                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON from props attribute: {e}")
                except Exception as e:
                    print(f"An error occurred while processing data: {e}")
            else:
                print("Error: astro-island element found, but no 'props' attribute.")
        else:
            print(f"Error: Could not find an astro-island element with component-url '{component_url_to_find}'.")
            print("Consider adjusting the selector or trying an alternative method (commented out in the script).")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching the URL: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


In [ ]:
test_ids = ["l8ugp31t","5ysod3tm"]
#,"h0x63109","l8re36n5"
player_data=[]
for id in test_ids:
    matches = get_Player_Events_Matches_from_URL(id)
    name = get_player_data_from_url(id)
    print(name)
    print(matches)
    L =[name + list(row) for row in matches.values]
    
    player_data.extend(L)

player_data_df = pd.DataFrame(player_data)
player_data_df.to_csv("complete_data_df.csv", index= False)
#print(player_data)

In [ ]:
import os
import pandas as pd
import time
import logging
import subprocess

# Set up logging
logging.basicConfig(filename='scrape_log2.txt', level=logging.INFO, format='%(asctime)s - %(message)s')
logging.info(f"Start log")

header_written = False


def load_existing_data(player_list_path, unique_ids_update_path):
    """Load existing data for players and return processed IDs"""
    unique_ids_update = pd.read_csv(unique_ids_update_path, header=None)
    if os.path.exists(player_list_path):
        unique_player_list = pd.read_csv(player_list_path, low_memory=False)
        processed_ids = set(unique_player_list['ID'].unique())
        global header_written
        header_written = True
    else:
        unique_player_list = pd.DataFrame()
        processed_ids = set()
    
    return unique_ids_update, processed_ids

def run_git_lfs_prune():
    """Run git lfs prune to remove old LFS files"""
    try:
        logging.info("Running git lfs prune to clean up LFS storage...")
        # Run the git lfs prune command with shell=True for Windows compatibility
        subprocess.run("git lfs prune", check=True, shell=True)
        logging.info("git lfs prune completed successfully.")
    except subprocess.CalledProcessError as e:
        logging.error(f"Failed to run git lfs prune: {e}")
        print(f"Error running git lfs prune: {e}")

def save_to_csv(player_list_path, player_data_list, header=False):
    """Save player data to CSV"""
    if player_data_list:
        try:
            player_df = pd.DataFrame(player_data_list)
            player_df.to_csv(player_list_path, mode='a', header=header, index=False)
            global header_written
            header_written = True
        except OSError as e:
            # If the error is due to disk space (Errno 28: No space left on device)
            if e.errno == 28:
                logging.error("Disk space full. Running git lfs prune...")
                print("Disk space full. Running git lfs prune...")
                run_git_lfs_prune()
                # Retry saving after running git lfs prune
                try:
                    player_df.to_csv(player_list_path, mode='a', header=header, index=False)
                    logging.info("Data saved successfully after running git lfs prune.")
                except Exception as e:
                    logging.error(f"Failed to save data after pruning: {e}")
                    print(f"Failed to save data after pruning: {e}")
            else:
                logging.error(f"Failed to save data: {e}")
                print(f"Failed to save data: {e}")


def process_players_in_batches(unique_ids_update, processed_ids, player_list_path, batch_size=50):
    """Process players in batches and save data periodically"""
    player_data = []
    to_process = [pid for pid in unique_ids_update[0] if pid not in processed_ids]
    total_batches = len(to_process) // batch_size + (1 if len(to_process) % batch_size != 0 else 0)

    try:
        for batch_idx in range(total_batches):
            start_idx = batch_idx * batch_size
            end_idx = min((batch_idx + 1) * batch_size, len(to_process))
            batch_ids = to_process[start_idx:end_idx]

            process_batch(batch_ids, player_data)

            # Save batch data
            save_to_csv(player_list_path, player_data, header=not header_written)
            
            print(f"Saved batch {batch_idx + 1}/{total_batches} after processing Player IDs {batch_ids}")
            logging.info(f"Saved batch {batch_idx + 1}/{total_batches} after processing Player IDs {batch_ids}")
            player_data = []  # Clear data for next batch

    except KeyboardInterrupt:
        # Handle graceful stop when interrupted (Ctrl+C)
        logging.info("Process interrupted. Saving current data before exit...")
        if player_data:
            save_to_csv(player_list_path, player_data, header=not header_written)
            logging.info(f"Saved remaining {len(player_data)} players before stopping.")
        print("Process interrupted. Current data saved.")

def process_batch(batch_ids, player_data):
    """Process each batch of player IDs"""
    for idx, id in enumerate(batch_ids):
        try:
            #fetch data
            matches = get_Player_Events_Matches_from_URL(id)
            name = get_player_data_from_url(id)
            #merge name and match data
            l = [name + list(row) for row in matches.values]
            player_data.extend(l)
            print(f"Index {idx}: Player {id} processed")
            time.sleep(0.1)  # Throttle requests

        except Exception as e:
            logging.error(f"Error processing Player ID {id}: {e}")
            print(f"Error processing Player ID {id}: {e}")
            continue  # Skip to the next player ID on error

        logging.info(f"Successfully processed Player ID {id}")

# Main processing flow
player_list_path = "UniquePlayerList2.csv"
unique_ids_update_path = "UniqueIds.csv"

# Load existing data
unique_ids_update, processed_ids = load_existing_data(player_list_path, unique_ids_update_path)

# Process players in batches and save data periodically
process_players_in_batches(unique_ids_update, processed_ids, player_list_path, batch_size=50)

print("Processing complete.")


Error processing Player ID ID: 'NoneType' object is not iterable
Index 1: Player uq3c0sod processed
Index 2: Player md1hfshh processed
Index 3: Player g4r98c2z processed
Index 4: Player hg2ipp29 processed
Index 5: Player o16qnc8p processed
Index 6: Player cnqxry9h processed
Index 7: Player s45soms9 processed
Index 8: Player ejapt8j9 processed
Index 9: Player 7rg50be1 processed
Index 10: Player 22e5wotg processed
Index 11: Player 93js2exg processed
Index 12: Player 34g161w8 processed
Index 13: Player basit3no processed
Index 14: Player hvr7xgus processed
Index 15: Player 820kab88 processed
Index 16: Player 1mkacvvw processed
Index 17: Player 0pvtbihd processed
Index 18: Player j7dln9w2 processed
Index 19: Player 3dh5j12s processed
Index 20: Player ak2rzeo0 processed
Index 21: Player fynmq672 processed
Index 22: Player vn7wxg9e processed
Index 23: Player bx8mutez processed
Index 24: Player xeb2501o processed
Index 25: Player 5498guwi processed
Index 26: Player c9c8s8ey processed
Index 27

In [18]:
#Split names and results


import pandas as pd
df = pd.read_csv("UniquePlayerList2.csv", low_memory=False)
print(df.head())
# Function to split result safely
def split_result(value):
    if isinstance(value, str) and (value.startswith("Won") or value.startswith("Lost") or value.startswith("Draw")):
        parts = value.split(' ', 1)
        return pd.Series([parts[0], parts[1]])
    elif value.startswith("ID"):
        return pd.Series(["ID", value])  # or ["ID", value] if you want to keep the ID

# Apply to column 6 (result column)
df[['match_result', 'match_score']] = df["result"].apply(split_result)
df = df.drop(columns=["result"])

# Split opponent name column 11
df[['opp_last_name', 'opp_first_name']] = df["opp_NAME"].str.split(', ', expand=True)
df = df.drop(columns=["opp_NAME"])



df.to_csv("UniquePlayerList2_splitnames_splitresults.csv", index=False)


         ID   lastname firstname    3  4    5    result tournamentcode  \
0  f917lein  Swarowski    Robert   13  1   14  Lost 0-0         ptla98   
1  f917lein  Swarowski    Robert  283  2  113   Won 0-0         ptla98   
2  f917lein  Swarowski    Robert  422  3   81   Won 0-0         ptla98   
3  f917lein  Swarowski    Robert  563  4   51  Lost 0-0         ptla98   
4  f917lein  Swarowski    Robert  771  5  103  Lost 0-0         ptla98   

        8       9    opp_ID            opp_NAME      12   13        date  \
0  1500.0  1482.0  7d05pfwn         Finkel, Jon  1500.0  8.0  1998-03-06   
1  1482.0  1500.0  1sgt9umm  De Almeida, Carlos  1482.0  8.0  1998-03-06   
2  1500.0  1518.0  j0ytuxye     Turning, Travis  1500.0  8.0  1998-03-06   
3  1518.0  1500.0  ryr6e6vq   Budzinski, Gerald  1518.0  8.0  1998-03-06   
4  1500.0  1482.0  2idl8bs4         Holcomb, Ty  1500.0  8.0  1998-03-06   

   is_pro             tournamentname  \
0    True  Pro Tour Los Angeles 1998   
1    True  Pro Tou

In [20]:

df_won = df[df["match_result"]=="Won"].copy()
df_lost = df[df["match_result"]=="Lost"].copy()
df_id = df[df["match_result"]=="ID"].copy()
df_draw = df[df["match_result"]=="Draw"].copy()
print(df_won.shape[0])
print(df_lost.shape[0])
print(df_draw.shape[0])
print(df_id.shape[0])


3015433
2722802
123743
20266


In [ ]:
import json
import csv

#create a list of all the tournaments from gatheringstats

# Load data
with open('GatheringStats_tournaments.json', 'r', encoding='utf-8') as f:
    players = json.load(f)

# Write csv
with open('GatheringStats_tournaments_medium_mapping.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=[
        "Tournament","name", "medium", "date", "coverage"
    ])
    writer.writeheader()
    for tournament, info in players.items():
        writer.writerow({
            "Tournament": tournament,
            "name":info.get("name","empty"),
            "medium": info.get("medium", "empty"),
            "date": info.get("date", "empty"),
            "coverage": info.get("coverage")
        })


In [1]:
#match the medium from gatheringstats to the listitems from eloproject

import pandas as pd
df_mediums = pd.read_csv("GatheringStats_tournaments_medium_mapping.csv", low_memory=False)
df_eloproject = pd.read_csv("UniquePlayerList2_splitnames_splitresults.csv", low_memory=False)

df_merged = pd.merge(df_eloproject,df_mediums,left_on="tournamentname",right_on="name", how="outer")
df_merged.fillna('unknown', inplace=True)

df_merged.to_csv("EloWithMedium.csv", index=False)



C:\Users\Jonathan\AppData\Local\Temp\ipykernel_25616\1778166255.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_merged.fillna('unknown', inplace=True)


Columns are:
ID,lastname,firstname,match_id,round,table,tournamentcode,own_elo_start,own_elo_end,opp_ID,opp_elo_start,cut_before,date_x,is_pro,tournamentname,coveragelink,tournamenttype,tournamentformat,match_result,match_score,opp_lastname,opp_firstname,tournamentcode_gathstat,tournamentname_gathstat,medium,date_y,coverage_gathstat

In [ ]:
#Separate all info that belongs to the players from the edges
#this means we have ID, firstname, lastname, is_pro as separate info
import pandas as pd
df = pd.read_csv("EloWithMedium.csv", low_memory=False)
df = df[(df['match_result'] == "Won") & (df['ID'] != 'unknown') & (df['opp_ID'] != 'unknown')]
# Select relevant columns and drop duplicate IDs
player_info = df[["ID", "firstname", "lastname", "is_pro"]].drop_duplicates(subset="ID")
player_info.to_csv("PlayerNodeData.csv", index=False)
df_check = df[~df['opp_ID'].isin(player_info["ID"])].drop_duplicates(subset="opp_ID")
df_check

,ID,lastname,firstname,match_id,round,table,tournamentcode,own_elo_start,own_elo_end,opp_ID,...,tournamentformat,match_result,match_score,opp_lastname,opp_firstname,tournamentcode_gathstat,tournamentname_gathstat,medium,date_y,coverage_gathstat
99,h6ke3hcc,Wolf,Andrew,1696.0,3,81.0,worlds98,1604.74,1584.19,6lb0rbqj,...,mixed,Lost,0-0,Alfar,Georgina,unknown,unknown,Tabletop,unknown,unknown
147,lewcamij,Ogilbee,Brian,2378.0,10,69.0,worlds98,1482.59,1500.3,3yrgdyif,...,mixed,Won,0-0,Ishibashi,Masashi,unknown,unknown,Tabletop,unknown,unknown
4077,l8ugp31t,Croft,Michael,14761.0,2,10.0,natca99,1505.01,1522.59,mgodx49r,...,mixed,Won,0-0,Lau,Angus,unknown,unknown,Tabletop,unknown,unknown
4141,u5fq048t,Charlish,Dany,14850.0,5,9.0,natca99,1464.02,1482.01,n0gtso36,...,mixed,Won,0-0,Otterson,Trevor,unknown,unknown,Tabletop,unknown,unknown
4194,ber2mot0,Radonjic,Peter,14820.0,4,9.0,natca99,1552.88,1569.92,84nmrycp,...,mixed,Won,0-0,Levy,Al,unknown,unknown,Tabletop,unknown,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5877173,gretkvjb,Fregosi,Lorenzo,3763976.0,1,77.0,rcpra25-open,1701.49,1715.87,bedo0cm1,...,modern,Won,2-0,SanG4,unknown,unknown,unknown,unknown,unknown,unknown
5877249,9apnens2,Takacs,Gabor,3765821.0,7,262.0,rcpra25-open,1363.74,1383.16,gnw1gko7,...,modern,Won,2-1,Kalashju,unknown,unknown,unknown,unknown,unknown,unknown
5877846,rs5f7ykk,Shales,Dave,3764160.0,1,263.0,rcpra25-open,1441.97,1461.02,pj4cl7ph,...,modern,Won,2-0,GamY1,unknown,unknown,unknown,unknown,unknown,unknown
5878621,rkh9c05q,Manigrasso,Marco,3764090.0,1,192.0,rcpra25-open,1500.0,1482.0,dr7i4un7,...,modern,Lost,1-2,Wengsk,unknown,unknown,unknown,unknown,unknown,unknown


In [24]:
df_id = pd.read_csv("UniqueIds.csv", low_memory=False)
df_diff = df_check[~df_check['opp_ID'].isin(df_id["ID"])].drop_duplicates(subset="opp_ID")
df_combine= pd.concat([df_id["ID"],df_diff["opp_ID"]], ignore_index=True)
df_combine.to_csv("UniqueIds.csv", index=False)

In [ ]:
''' 
This allows to filter for all entries that satisfy certain conditions and modify them
'''
import pandas as pd
import numpy as np
df = pd.read_csv("EloWithMedium.csv", low_memory=False)
#mask = (df['date_x'] < "2002-01-01") | (df['date_y'] < "2002-01-01")
mask = (df['coveragelink'] == "https://melee.gg/Tournament/View/203520") | (df['coveragelink'] == "https://web.archive.org/web/20210920205756/https://magic.wizards.com/en/events/coverage/2017natjp")

df.loc[mask, "medium"] = "Tabletop"
df.to_csv("EloWithMedium.csv",index=False)


In [ ]:
'''Filter out all instances where medium is unknown'''

import pandas as pd

df = pd.read_csv("EloWithMedium.csv", low_memory=False)
mask = (df['date_x'] > "2003-01-01") | (df['date_y'] > "2003-01-01")
df = df[df["medium"]=="unknown"]
#df = df[df['date_x'] > "2003-01-01"]
#df = df["coveragelink"].unique()
df.to_csv("TestData.csv", index=False)
print(df.shape[0])
print(df)


In [1]:
#create sample test set with n random entries
import pandas as pd
df = pd.read_csv("EloWithMedium.csv", low_memory=False)
#make sure only to pick entries with known contestants and only sample wins
df = df[(df['match_result'] == "Won") & (df['ID'] != 'unknown') & (df['opp_ID'] != 'unknown')]
df = df.sample(n=10000, random_state=1)
df.to_csv("Sample10000_n1.csv", index=False)

In [38]:
#k-core implementation for preprocessing
import pandas as pd

def to_k_core(df, core):
    while True:
        #count both occcurences of nodes as player and as opp
        counts = pd.concat([df['ID'], df['opp_ID']]).value_counts()
        to_keep = counts[counts >= core].index
        prev_len = len(df)
        df = df[df['ID'].isin(to_keep) & df['opp_ID'].isin(to_keep)]
        if len(df) == prev_len:
            break
    print(df['ID'].value_counts())
    print(df)
    return df

In [46]:
import pandas as pd

df = pd.read_csv("Sample10000_n1.csv", low_memory=False)
df = to_k_core(df, 2)
df.to_csv("KCoreTest.csv", index=False)

ID
6bi207s2    3
r4108x35    3
u9zedek0    2
ho8e03qi    2
hwih6mgh    2
9ufkflg6    2
z86z8qv1    2
apxvwg9g    2
y0mwk2ye    2
otidul2z    2
870wn05h    2
bmkubhg4    1
ya3lmeq1    1
0z5g08kc    1
aroxbjeq    1
rbfnurpp    1
lmqkl48t    1
7ysd3tvc    1
q3qt15fi    1
k4q6h6om    1
i42eqj07    1
ccwsbbsz    1
oz33n7wy    1
e4x33lyi    1
du9lfioc    1
esocbna4    1
w2mq3pye    1
i8olnjut    1
grfz9457    1
9lh9td82    1
0ifl6t22    1
y3utfgv1    1
wr1ssrjg    1
hsb2nxmz    1
ef6cwyk8    1
oe4o0i3r    1
3m2v9pq5    1
h44y3n2c    1
ore6tgie    1
9mrkq87r    1
2eefgein    1
akq5t76h    1
Name: count, dtype: int64
            ID      lastname       firstname          3   4      5  \
15    rbfnurpp          Ruel         Olivier   514330.0   2  129.0   
100   grfz9457         Bunko         Matthew  2088175.0   5   13.0   
358   2eefgein  Utter-Leyton            Josh  3443732.0   7    5.0   
408   9ufkflg6        Austin           Lance  3049585.0   6   23.0   
562   9mrkq87r         Dugal  Lou

In [55]:
''' 
Convert the dataset to a different format so it can be used with other libraries, to calc k-core etc
'''
import networkx as nx
import pandas as pd
# load data
df = pd.read_csv("Sample10000_n1.csv", low_memory=False)

# convert to nx graph
G = nx.from_pandas_edgelist(df, source='ID', target='opp_ID', edge_attr=True, create_using=nx.MultiDiGraph())
# Convert to undirected simple graph and get k-core node set
G_undirected_simple = nx.Graph(G.to_undirected())
core_nodes = set(nx.k_core(G_undirected_simple, k=2).nodes())
# Filter original directed graph to only keep nodes in k-core
G_core_directed = G.subgraph(core_nodes).copy()
# Now G_core_directed is the filtered directed graph
nx.write_gexf(G_core_directed, "test.gexf")

# Filter the DataFrame to only keep edges where both nodes are in the k-core
df_kcore = df[df['ID'].isin(core_nodes) & df['opp_ID'].isin(core_nodes)]
df_kcore
# Output as edgelist
#core_edges = nx.to_pandas_edgelist(G_core_directed)
df_kcore.to_csv("Sample10000_n1_kcore_2.csv", index=False)
